# Web Scraping

## installing selenium in colab

In [22]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("download.default_directory=/content")
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromiu

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]


## Step 1: Download College directory listing using Selenium and Chrome

In [23]:
from selenium.webdriver.support.ui import Select
# A funtion to select correct options and download data
def download(url):
    # Passing URL to browser
    driver.get(url)
    # selecting EXCEL from dropdown list 
    select = Select(driver.find_element_by_name("reportFormatType"))
    select.select_by_value('2')
    # Selecting the "search button"
    button = driver.find_element_by_class_name("btn-primary")
    button.click()
    
url = f"http://aishe.nic.in/aishe/collegeDirectoryIndex?hasReportLink=index"
download(url)

## Step 2: Read and list website URL into a different file.
Expecting all three files downloaded earlier to be there in the uni_data folder.

In [29]:
import glob, os
os.chdir("/content")
excelfile = ''
for file in glob.glob("*.xlsx"):
  excelfile = file

In [30]:
import os
(_, _, filenames) = next(os.walk("/content"))
import pandas as pd
# loading the file to a pandas dataframe
df = pd.read_excel(f"/content/{excelfile}",header=2,skiprows=[3,3])
df = df[['College Name','Address','Website']]

In [31]:
# renaming columns
df.rename(columns={'College Name':'name','Address':'address','Website':'website'},inplace=True)
df.head(5)

,name,address,website
0,Jawaharlal Nehru Rajkeeya Mahavidyalaya (Id: ...,"Jawaharlal Nehru Rajkeeya Mahavidyalaya, M S ...",jnrm.and.nic.in
1,Regional Medical Research Institute (I.C.M.R.)...,"Post Bag No 13, Dollygunj, Port Blair",http://www.rmrc.res.in
2,Tagore Government College of Education (Id: C-...,"M.G. Road, Middle Point, Port Blair - 744101",www.tgce.and.nic.in
3,Zoological Survey of India (Id: C-6560),"Zoological Survey of India, Andaman & Nicobar ...",www.zsi.gov.in
4,Mahatma Gandhi Govt. College (Id: C-6547),"Rampur, Mayabunder, Mayabunder",http://mggcm.and.nic.in


In [32]:
# adding new columns to df-copy
dfc = df.copy()
#clean url from df
def clean_url(url):
    if isinstance(url, str): 
        if (url.startswith("http://") or url.startswith("https://")):
            return url
        else:
            return "http://"+url
    else:
        return 'nan'

# cleaning URL's
dfc["website"] = dfc.apply(lambda row : clean_url(row['website']), axis=1)

# save file
dfc.to_csv("cleanedOriginal.csv")

## Step 3: Loop through URLs and scrap data.
#### Add Email, Contact No to the df

In [36]:
# imp package imports
import itertools
# possible url extensions
dir_lv1 = ['/Contact/','/Contact-us/','/ContactUs/','/Contact_us/','/']
dir_lv2 = ['contact','contact_us','contact-us','contactus']
# possible file extensions
str1 = ".ASHX .ASP .ASPX .HTM .HTML .HTMLS .HXS .JSP .JSPX .MSPX .PHP .PHP3 .WSDL .XHTML"
formats_i = str1.split()
formats_i.extend([''])
# list all url combinations
def combinator(l1, l2, l3):
    comb_0=[]
    for i in l1:
        for j in l2:
            for k in l3:
                comb_0.append(i+j+k)
    return comb_0
T1_comb = combinator(dir_lv1, dir_lv2, formats_i)
contactno = []
emailid = []

In [45]:
import requests
import copy
# scrape webpage and extract email ID
def emailscrap(url, comb):
    el1 = []
    nmURL = copy.deepcopy(url)
    for i in comb:
        print(nmURL)
        try:
            response = requests.get(nmURL)
            nmURL = url+i
        except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError):
            # ignore pages with errors and continue with next url
            nmURL = url+i
            continue
        new_emails = set(re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.com", response.text, re.I)) # re.I: (ignore case)
        el1.extend(new_emails)
    return ", ".join(el1)

In [ ]:
# testing!
emailscrap("http://jnrm.and.nic.in", T1_comb)

http://jnrm.and.nic.in
http://jnrm.and.nic.in/Contact/contact.ASHX
http://jnrm.and.nic.in/Contact/contact.ASP
http://jnrm.and.nic.in/Contact/contact.ASPX
http://jnrm.and.nic.in/Contact/contact.HTM
http://jnrm.and.nic.in/Contact/contact.HTML
http://jnrm.and.nic.in/Contact/contact.HTMLS
http://jnrm.and.nic.in/Contact/contact.HXS
http://jnrm.and.nic.in/Contact/contact.JSP
http://jnrm.and.nic.in/Contact/contact.JSPX
http://jnrm.and.nic.in/Contact/contact.MSPX
http://jnrm.and.nic.in/Contact/contact.PHP
http://jnrm.and.nic.in/Contact/contact.PHP3
http://jnrm.and.nic.in/Contact/contact.WSDL
http://jnrm.and.nic.in/Contact/contact.XHTML
http://jnrm.and.nic.in/Contact/contact
http://jnrm.and.nic.in/Contact/contact_us.ASHX
http://jnrm.and.nic.in/Contact/contact_us.ASP
http://jnrm.and.nic.in/Contact/contact_us.ASPX
http://jnrm.and.nic.in/Contact/contact_us.HTM
http://jnrm.and.nic.in/Contact/contact_us.HTML
http://jnrm.and.nic.in/Contact/contact_us.HTMLS
http://jnrm.and.nic.in/Contact/contact_us.HX

In [ ]:
#
dft = dfc.copy()
dft["emails"] = dft.apply(lambda row : emailscrap(row['website'], T1_comb), axis=1)